In [1]:
import torch
from torch.utils.data import Dataset
from torchvision import datasets
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from torch import nn
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt
import numpy as np
import tensorboard
import torchvision
from torch.utils.tensorboard import SummaryWriter
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))
from sklearn.model_selection import KFold

Using cuda device


In [2]:
transform = transforms.Compose(
            [transforms.Resize([32, 32]),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])

In [3]:
train_dataset = datasets.ImageFolder(root='D:/split_data_breast/train', 
                                     transform=transform)
test_dataset  = datasets.ImageFolder(root='D:/split_data_breast/test',  
                                     transform=transform)
valid_dataset  = datasets.ImageFolder(root='D:/split_data_breast/val',  
                                     transform=transform)

In [27]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_dataset, batch_size=96, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=96, shuffle=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=96, shuffle=True)

In [28]:
import torch 
import torch.nn as nn
import torchvision.models as models
#resnet18 = models.resnet18(pretrained=True)
alexnet = models.alexnet(pretrained=True)
vgg16 = models.vgg16(pretrained=True)
#squeezenet = models.squeezenet1_0(pretrained=True)
densenet = models.densenet161(pretrained=True)
#inception = models.inception_v3()

In [29]:
densenet

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 96, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(96, 192, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(192, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(144, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (rel

In [30]:


for param in densenet.parameters():
    param.requires_grad = False
    
densenet.classifier = nn.Linear(2208,2)


In [31]:
densenet

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 96, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(96, 192, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(192, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(144, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (rel

In [32]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [33]:
def train_model_and_test(model, learning_rate):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
    model.to(device)
    print('Starting training with learning rate = '+ str(learning_rate))
    for epoch in range(25):
        running_loss = 0.0
        for i, data in enumerate(train_dataloader, 0):
            inputs, labels = data[0].to(device), data[1].to(device)

            optimizer.zero_grad()

            output = model(inputs)
            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            if i % 50 == 49:
                print('[%d, %5d] loss: %.3f' %(epoch + 1, i + 1, running_loss / 50))
                running_loss = 0.0

    print('Finished Training the model')
    
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_dataloader:
            images, labels = data[0].to(device), data[1].to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accr = (100 * correct / total)
    print('Accuracy of the model network: %d %%' % (accr))
    return accr


In [34]:
import torch.optim as optim
train_model_and_test(densenet, 0.01)

Starting training with learning rate = 0.01
[1,    50] loss: 0.480
[1,   100] loss: 0.431
[1,   150] loss: 0.488
[1,   200] loss: 0.461
[1,   250] loss: 0.438
[1,   300] loss: 0.455
[1,   350] loss: 0.450
[1,   400] loss: 0.432
[1,   450] loss: 0.446
[1,   500] loss: 0.453
[1,   550] loss: 0.437
[1,   600] loss: 0.445
[1,   650] loss: 0.450
[1,   700] loss: 0.443
[1,   750] loss: 0.444
[1,   800] loss: 0.426
[1,   850] loss: 0.442
[1,   900] loss: 0.449
[1,   950] loss: 0.443
[1,  1000] loss: 0.446
[1,  1050] loss: 0.478
[1,  1100] loss: 0.477
[1,  1150] loss: 0.469
[1,  1200] loss: 0.431
[1,  1250] loss: 0.457
[1,  1300] loss: 0.443
[1,  1350] loss: 0.468
[1,  1400] loss: 0.433
[1,  1450] loss: 0.458
[1,  1500] loss: 0.464
[1,  1550] loss: 0.427
[1,  1600] loss: 0.434
[1,  1650] loss: 0.437
[1,  1700] loss: 0.417
[1,  1750] loss: 0.472
[1,  1800] loss: 0.458
[1,  1850] loss: 0.440
[1,  1900] loss: 0.454
[1,  1950] loss: 0.432
[1,  2000] loss: 0.435
[1,  2050] loss: 0.488
[1,  2100] lo

82.18699333453432